# The Cancer Gene Atlas Cancer (TCGA) - Glioblastoma Multiforme (GBM)

Author: Moshe Silverstein <br>
Date: 7-17 <br>
Data Source: https://cancergenome.nih.gov/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

In [3]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [4]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/TCGA/untility_functions.py'>

# Versions Of Modules In Use

In [5]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget, seaborn 

Software versions
Python 3.5.2 64bit [GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
IPython 5.3.0
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.13.1
pandas 0.21.0
clustergrammer_widget 1.9.0
seaborn 0.7.1
Thu Apr 26 15:30:50 2018 EDT

# Functions

In [6]:
def getGenes(inputDF):
    
    inputDF.reset_index(inplace=True)
    
    inputDF.rename(columns={'index':'Gene Symbol'}, inplace=True)

    inputDF.set_index('Gene Symbol', inplace=True)

    lst = []

    for index in  inputDF.index:
        lst.append(index.split('|')[5])

    inputDF.index = lst

In [7]:
def getSampleID(inputDF, metaDF):
    
    lst = []
    
    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()

        ID = ('-').join(metaDF.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        
        ID = ID+'-'+metaDF.loc[col, 'a_AliquotBarcode'].split('-')[3][0:2]

        lst.append(ID)

        
    inputDF.columns = lst

In [8]:
def getSampleData(inputDF, metaDF, SampleTypes):
    
    lst = []

    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()
        
#         ID = ('-').join(meta.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        ID = ('-').join(col.split('-')[0:3])
        
        donor = 'Donor:'+ID
        
#         if '01' in str(col.split('-')[3]):
#             sampleType = 'Primary Solid Tumor'
#         elif '11' in str(col.split('-')[3]):
#             sampleType= 'Solid Tissue Normal'
        
        if ID in metaDF.index:
            info = (donor, 
                'Age:'+str(metaDF.loc[ID, 'age_at_initial_pathologic_diagnosis']),
                'Gender:'+str(metaDF.loc[ID, 'gender']),    
                'SampleType:'+SampleTypes[str(col.split('-')[3])],
                'Stage:'+str(metaDF.loc[ID, 'pathologic_stage']),
                'Histology:'+str(metaDF.loc[ID, 'histological_type']),
                'Tissue:'+str(metaDF.loc[ID, 'tumor_tissue_site']), 
                'Tumor:'+str(metaDF.loc[ID, 'tumor_type']),    
                'Grade'+str(metaDF.loc[ID, 'neoplasm_histologic_grade'])
#                 'SampleType:'+str(ID.split('-')[3])
               )
#             for col in metaDF.columns:
#                 if len(metaDF[col].dropna()) >= .5*len(metaDF[col]) and len(metaDF[col].unique()) > 2 and len(metaDF[col].unique()) < len(metaDF[col]) and 'age' not in col:
#                     info = info + (str(col)+':'+str(metaDF.loc[ID, col]),)
        lst.append(info)

        
    inputDF.columns = lst

In [9]:
SampleTypes ={
    '01':'Primary Solid Tumor',
    '02':'Recurrent Solid Tumor',
    '03':'Primary Blood Derived Cancer - Peripheral Blood',
    '04':'Recurrent Blood Derived Cancer - Bone Marrow',
    '05':'Additional - New Primary',
    '06':'Metastatic',
    '07':'Additional Metastatic',
    '08':'Human Tumor Original Cells',
    '09':'Primary Blood Derived Cancer - Bone Marrow',
    '10':'Blood Derived Normal',
    '11':'Solid Tissue Normal',
    '12':'Buccal Cell Normal',
    '13':'EBV Immortalized Normal',
    '14':'Bone Marrow Normal',
    '15':'sample type 15',
    '16':'sample type 16',
    '20':'Control Analyte',
    '40':'Recurrent Blood Derived Cancer - Peripheral Blood',
    '50':'Cell Lines',
    '60':'Primary Xenograft Tissue',
    '61':'Cell Line Derived Xenograft Tissue',
    '99':'sample type 99'
             }

# Path to Output Files

In [10]:
path = '/Users/moshesilverstein/Documents/Harmonizome/TCGA/Output/'

# Load Meta Data

In [11]:
meta = pd.read_csv('Input/TCGA_Metadata.csv', index_col=9)

In [12]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


In [13]:
meta.shape

(11373, 9)

# Load Sample MetaData

In [14]:
sample_meta = pd.read_csv('Input/metadata_Glioblastoma multiforme [GBM]', sep='\t', index_col=1)

In [15]:
sample_meta.head()

,Unnamed: 0,Study,Project,ParticipantUUID,TSSCode,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision,batch_number,bcr,clinical_M,...,BMI,age_began_smoking_in_years,h_pylori_infection,other_dx,other_malignancy_anatomic_site,other_malignancy_histological_type,other_malignancy_malignancy_type,stopped_smoking_year,venous_invasion,year_of_tobacco_smoking_onset
ParticipantBarcode,,,,,,,,,,,,,,,,,,,,,
TCGA-06-A7TK,9799,GBM,TCGA,41685C5A-A548-483A-8A20-305AD8D61771,06,64.0,NaN,402,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-RR-A6KA,9807,GBM,TCGA,225F3689-221D-4296-8472-D8C21EEDAF8D,RR,72.0,NaN,346,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-RR-A6KB,9808,GBM,TCGA,6F1B1011-D7A4-4A05-B6FF-288183E7254B,RR,51.0,NaN,346,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-32-1986,9809,GBM,TCGA,7ac80045-7c05-4e8f-8217-a380544a6125,32,68.0,NaN,26,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-OX-A56R,9816,GBM,TCGA,0FC32169-8EB9-405F-BECC-ED38D7064A58,OX,68.0,NaN,346,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
sample_meta.shape

(596, 70)

# Load Data

In [17]:
matrix = pd.read_csv('Input/TCGA_GBM_tpm.tsv', sep='\t', index_col=0)

In [18]:
matrix.head()

,8ef04a61-1ee2-482f-8647-e5812772d75e,db7b5149-a664-4c21-b12e-3fa5da0adbe4,9f5bfbba-f35d-4b69-8332-90aaf0686341,a5d01848-b248-480f-ad14-6655ddbe62a9,0b0b7b0c-d669-4667-8b3f-39c4c2946bbb,cbcbb5f5-03a4-4862-bf7f-480d9d3a9aeb,5fc61377-0c2c-4d92-b774-3ceb495a2cb4,88f0b526-1ff9-4439-ad3e-82c5d6aab951,d02bb6e1-5202-47f7-a567-18be4064ce1c,dfbe5a79-1a54-4b05-b41b-727313550540,...,32327714-12aa-4593-ad42-8b13dd4ea56f,a1dde794-445d-43d5-9b22-c159e9151714,8aad77b9-1ab6-4e91-b9df-25c928a3cd9c,2e52bc09-1ac8-41ec-b0f0-31330bd4cf42,52918733-0b2d-462b-8784-70bd148d9ffe,4b87419b-e54d-4173-9dfd-de51f5ddf531,6c2f908d-4c9a-49ef-8309-7429a196cc43,f688e89b-25aa-44e1-adf8-c996e3a78907,6707171e-29a2-44a3-9c6b-72b393cbedb3,a3682a7e-47ae-4125-b4d7-7d340bcfdbff
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,291.933000,358.43200,245.634000,158.17100,271.401000,10.632000,178.355000,135.798000,148.910000,221.86200,...,133.722000,153.101000,170.800000,216.28200,284.6050,194.3310,167.3580,177.77400,119.19900,146.0440
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,24.238200,30.63120,27.269800,38.09300,33.090600,0.484789,22.433300,20.827500,38.735500,27.03990,...,44.867800,60.344800,44.387400,34.49780,20.8830,23.5815,27.7558,27.51870,26.26090,32.8225
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,20.684100,12.58220,17.033300,10.79030,15.920300,0.000000,15.928800,11.423300,18.814800,16.76220,...,18.026800,15.606600,11.223800,7.45048,16.4109,13.8413,9.5998,13.42530,15.84490,14.1111
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,40.076100,52.83910,28.517000,42.55840,41.933700,1.890940,25.036600,21.527000,29.569300,64.79480,...,29.796800,35.372000,55.770000,29.91820,44.5655,62.2780,45.3860,58.57780,63.01300,67.2933
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.209828,3.54581,0.037845,1.23608,0.254702,0.037522,0.149606,0.262386,0.176437,1.00638,...,0.331268,0.101335,0.609697,0.00000,0.0000,4.6928,10.6790,5.45796,2.01177,11.4371


In [19]:
matrix.shape

(199169, 175)

# Get Only Protein Coding Genes

In [20]:
lst = []

for i,index in enumerate(matrix.index):
    
    progressPercent = ((i+1)/len(matrix.index)*100)

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(matrix.index)))
    sys.stdout.flush()
                                 
    lst.append(index.split('|')[7])    
                       
matrix['Gene Type'] = lst

matrix = matrix[matrix['Gene Type'] == 'protein_coding']

matrix.drop('Gene Type', axis=1, inplace=True)

In [21]:
matrix.head()

,8ef04a61-1ee2-482f-8647-e5812772d75e,db7b5149-a664-4c21-b12e-3fa5da0adbe4,9f5bfbba-f35d-4b69-8332-90aaf0686341,a5d01848-b248-480f-ad14-6655ddbe62a9,0b0b7b0c-d669-4667-8b3f-39c4c2946bbb,cbcbb5f5-03a4-4862-bf7f-480d9d3a9aeb,5fc61377-0c2c-4d92-b774-3ceb495a2cb4,88f0b526-1ff9-4439-ad3e-82c5d6aab951,d02bb6e1-5202-47f7-a567-18be4064ce1c,dfbe5a79-1a54-4b05-b41b-727313550540,...,32327714-12aa-4593-ad42-8b13dd4ea56f,a1dde794-445d-43d5-9b22-c159e9151714,8aad77b9-1ab6-4e91-b9df-25c928a3cd9c,2e52bc09-1ac8-41ec-b0f0-31330bd4cf42,52918733-0b2d-462b-8784-70bd148d9ffe,4b87419b-e54d-4173-9dfd-de51f5ddf531,6c2f908d-4c9a-49ef-8309-7429a196cc43,f688e89b-25aa-44e1-adf8-c996e3a78907,6707171e-29a2-44a3-9c6b-72b393cbedb3,a3682a7e-47ae-4125-b4d7-7d340bcfdbff
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,291.933000,358.43200,245.634000,158.17100,271.401000,10.632000,178.355000,135.798000,148.910000,221.86200,...,133.722000,153.101000,170.800000,216.28200,284.6050,194.3310,167.3580,177.77400,119.19900,146.0440
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,24.238200,30.63120,27.269800,38.09300,33.090600,0.484789,22.433300,20.827500,38.735500,27.03990,...,44.867800,60.344800,44.387400,34.49780,20.8830,23.5815,27.7558,27.51870,26.26090,32.8225
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,20.684100,12.58220,17.033300,10.79030,15.920300,0.000000,15.928800,11.423300,18.814800,16.76220,...,18.026800,15.606600,11.223800,7.45048,16.4109,13.8413,9.5998,13.42530,15.84490,14.1111
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,40.076100,52.83910,28.517000,42.55840,41.933700,1.890940,25.036600,21.527000,29.569300,64.79480,...,29.796800,35.372000,55.770000,29.91820,44.5655,62.2780,45.3860,58.57780,63.01300,67.2933
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.209828,3.54581,0.037845,1.23608,0.254702,0.037522,0.149606,0.262386,0.176437,1.00638,...,0.331268,0.101335,0.609697,0.00000,0.0000,4.6928,10.6790,5.45796,2.01177,11.4371


In [22]:
matrix.shape

(79930, 175)

# Map Sample Meta Data to Sample ID

In [23]:
getSampleID(matrix, meta)

In [24]:
matrix.head()

,TCGA-32-1970-01,TCGA-28-5218-01,TCGA-19-2619-01,TCGA-32-2634-01,TCGA-06-5415-01,TCGA-26-5134-01,TCGA-32-1982-01,TCGA-19-2620-01,TCGA-32-2638-01,TCGA-06-5416-01,...,TCGA-14-0789-01,TCGA-16-1045-01,TCGA-27-1834-01,TCGA-14-2554-01,TCGA-27-2523-01,TCGA-06-0681-11,TCGA-06-0675-11,TCGA-06-0680-11,TCGA-06-AABW-11,TCGA-06-0678-11
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,291.933000,358.43200,245.634000,158.17100,271.401000,10.632000,178.355000,135.798000,148.910000,221.86200,...,133.722000,153.101000,170.800000,216.28200,284.6050,194.3310,167.3580,177.77400,119.19900,146.0440
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,24.238200,30.63120,27.269800,38.09300,33.090600,0.484789,22.433300,20.827500,38.735500,27.03990,...,44.867800,60.344800,44.387400,34.49780,20.8830,23.5815,27.7558,27.51870,26.26090,32.8225
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,20.684100,12.58220,17.033300,10.79030,15.920300,0.000000,15.928800,11.423300,18.814800,16.76220,...,18.026800,15.606600,11.223800,7.45048,16.4109,13.8413,9.5998,13.42530,15.84490,14.1111
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,40.076100,52.83910,28.517000,42.55840,41.933700,1.890940,25.036600,21.527000,29.569300,64.79480,...,29.796800,35.372000,55.770000,29.91820,44.5655,62.2780,45.3860,58.57780,63.01300,67.2933
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.209828,3.54581,0.037845,1.23608,0.254702,0.037522,0.149606,0.262386,0.176437,1.00638,...,0.331268,0.101335,0.609697,0.00000,0.0000,4.6928,10.6790,5.45796,2.01177,11.4371


# Map Gene Symbol to Gene ID

In [25]:
getGenes(matrix)

In [26]:
matrix.head()

,TCGA-32-1970-01,TCGA-28-5218-01,TCGA-19-2619-01,TCGA-32-2634-01,TCGA-06-5415-01,TCGA-26-5134-01,TCGA-32-1982-01,TCGA-19-2620-01,TCGA-32-2638-01,TCGA-06-5416-01,...,TCGA-14-0789-01,TCGA-16-1045-01,TCGA-27-1834-01,TCGA-14-2554-01,TCGA-27-2523-01,TCGA-06-0681-11,TCGA-06-0675-11,TCGA-06-0680-11,TCGA-06-AABW-11,TCGA-06-0678-11
ARF5,291.933000,358.43200,245.634000,158.17100,271.401000,10.632000,178.355000,135.798000,148.910000,221.86200,...,133.722000,153.101000,170.800000,216.28200,284.6050,194.3310,167.3580,177.77400,119.19900,146.0440
M6PR,24.238200,30.63120,27.269800,38.09300,33.090600,0.484789,22.433300,20.827500,38.735500,27.03990,...,44.867800,60.344800,44.387400,34.49780,20.8830,23.5815,27.7558,27.51870,26.26090,32.8225
ESRRA,20.684100,12.58220,17.033300,10.79030,15.920300,0.000000,15.928800,11.423300,18.814800,16.76220,...,18.026800,15.606600,11.223800,7.45048,16.4109,13.8413,9.5998,13.42530,15.84490,14.1111
FKBP4,40.076100,52.83910,28.517000,42.55840,41.933700,1.890940,25.036600,21.527000,29.569300,64.79480,...,29.796800,35.372000,55.770000,29.91820,44.5655,62.2780,45.3860,58.57780,63.01300,67.2933
CYP26B1,0.209828,3.54581,0.037845,1.23608,0.254702,0.037522,0.149606,0.262386,0.176437,1.00638,...,0.331268,0.101335,0.609697,0.00000,0.0000,4.6928,10.6790,5.45796,2.01177,11.4371


# Histogram of First Sample

In [27]:
sns.distplot(matrix.iloc[:, 0]);

# Histogram of First Gene

In [28]:
sns.distplot(matrix.iloc[0, :]);

# Save Unfiltered Matrix to File

In [29]:
filename = path+'tcga_gbm_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Filtered

In [30]:
normalized_matrix = matrix.copy()

In [31]:
normalized_matrix = normalized_matrix.groupby(level=0).max()

# Remove any data with more than 95% missing and imput remaining mssing values to matrix mean

In [32]:
normalized_matrix = uf.removeAndImpute(normalized_matrix)

In [33]:
normalized_matrix.head()

,TCGA-32-1970-01,TCGA-28-5218-01,TCGA-19-2619-01,TCGA-32-2634-01,TCGA-06-5415-01,TCGA-26-5134-01,TCGA-32-1982-01,TCGA-19-2620-01,TCGA-32-2638-01,TCGA-06-5416-01,...,TCGA-14-0789-01,TCGA-16-1045-01,TCGA-27-1834-01,TCGA-14-2554-01,TCGA-27-2523-01,TCGA-06-0681-11,TCGA-06-0675-11,TCGA-06-0680-11,TCGA-06-AABW-11,TCGA-06-0678-11
A1BG,0.112039,0.140683,0.140683,0.140683,0.256106,0.052575,0.186558,0.212980,0.150423,0.140683,...,0.140683,0.140683,0.140769,0.279762,0.140683,0.140683,0.140683,0.140683,0.034853,0.140683
A1CF,0.012448,0.025190,0.024174,0.016166,0.015305,0.001462,0.015144,0.016213,0.005859,0.030789,...,0.004606,0.015144,0.004075,0.009078,0.010094,0.011281,0.020323,0.033674,0.031317,0.015806
A2M,173.929000,656.984000,366.005000,330.287000,170.065000,10.447400,272.946000,304.840000,390.553000,115.702000,...,854.171000,365.734000,307.176000,137.330000,181.250000,79.689400,88.940300,83.646000,73.356800,62.172300
A2ML1,2.665070,0.121547,0.611536,1.537490,1.206310,0.020419,0.220810,2.233830,0.220992,0.083328,...,0.260359,0.758004,3.221620,0.892674,0.204329,0.484600,0.787420,1.170380,0.366623,0.411216
A3GALT2,0.144533,0.107838,0.061480,0.324748,0.020509,0.235753,0.187242,0.329740,0.180570,0.411337,...,0.235753,0.127061,0.235753,0.047833,0.164123,0.074316,0.108028,0.105512,0.128227,0.029574


In [34]:
normalized_matrix.shape

(18992, 175)

# logCPM Transform

Raw counts were normalized to log10-Counts Per Million (logCPM) by dividing each column by the total sum of its counts, multiplying it by 106, followed by the application of a log10-transform.

In [35]:
samples = normalized_matrix.columns.tolist()

In [36]:
normalized_matrix.columns = np.arange(0, len(normalized_matrix.columns))

In [37]:
for col in normalized_matrix.columns:

    total = normalized_matrix[col].sum()
    
    normalized_matrix[col] = normalized_matrix[col].apply(lambda x: np.log10(((x)/total)*10**6))

In [38]:
normalized_matrix.columns = samples

In [39]:
normalized_matrix.head()

,TCGA-32-1970-01,TCGA-28-5218-01,TCGA-19-2619-01,TCGA-32-2634-01,TCGA-06-5415-01,TCGA-26-5134-01,TCGA-32-1982-01,TCGA-19-2620-01,TCGA-32-2638-01,TCGA-06-5416-01,...,TCGA-14-0789-01,TCGA-16-1045-01,TCGA-27-1834-01,TCGA-14-2554-01,TCGA-27-2523-01,TCGA-06-0681-11,TCGA-06-0675-11,TCGA-06-0680-11,TCGA-06-AABW-11,TCGA-06-0678-11
A1BG,-0.736487,-0.666846,-0.644772,-0.626708,-0.383086,0.169118,-0.533712,-0.484727,-0.612254,-0.644909,...,-0.666312,-0.655897,-0.653694,-0.362225,-0.660049,-0.646653,-0.669305,-0.660575,-1.215596,-0.639542
A1CF,-1.690760,-1.413852,-1.409659,-1.566336,-1.606664,-1.386666,-1.624287,-1.603200,-2.021750,-1.304754,...,-2.151191,-1.623898,-2.192050,-1.851023,-1.804236,-1.742555,-1.509563,-1.281525,-1.262052,-1.588972
A2M,2.454516,3.002467,2.770474,2.743942,2.439109,2.467350,2.631550,2.671006,2.802111,2.270190,...,3.116992,2.759027,2.685186,2.328751,2.449988,2.106506,2.131552,2.113629,2.107603,2.005814
A2ML1,0.639853,-0.730343,-0.006591,0.411863,0.289953,-0.241615,-0.460508,0.535984,-0.445192,-0.872362,...,-0.398981,0.075533,0.705873,0.141679,-0.497960,-0.109511,0.078660,0.259511,-0.193618,-0.173713
A3GALT2,-0.625889,-0.782316,-1.004281,-0.263403,-1.479565,0.820799,-0.532123,-0.294894,-0.532923,-0.178953,...,-0.442096,-0.700126,-0.429744,-1.129288,-0.593121,-0.923814,-0.784010,-0.785514,-0.649858,-1.316868


# Normalize Matrix (Quantile Normalize the matrix for the columns)

In [40]:
normalized_matrix = uf.quantileNormalize(normalized_matrix)

In [41]:
normalized_matrix.head()

,TCGA-32-1970-01,TCGA-28-5218-01,TCGA-19-2619-01,TCGA-32-2634-01,TCGA-06-5415-01,TCGA-26-5134-01,TCGA-32-1982-01,TCGA-19-2620-01,TCGA-32-2638-01,TCGA-06-5416-01,...,TCGA-14-0789-01,TCGA-16-1045-01,TCGA-27-1834-01,TCGA-14-2554-01,TCGA-27-2523-01,TCGA-06-0681-11,TCGA-06-0675-11,TCGA-06-0680-11,TCGA-06-AABW-11,TCGA-06-0678-11
A1BG,-0.830740,-0.576513,-0.635047,-0.772706,-0.433349,-0.352508,-0.599427,-0.508627,-0.567800,-0.666652,...,-0.615912,-0.692924,-0.770129,-0.236089,-0.486366,-0.718372,-0.747956,-0.764845,-1.290125,-0.744514
A1CF,-1.725661,-1.383851,-1.401902,-1.625769,-1.531900,-1.727615,-1.788268,-1.676858,-2.387875,-1.330799,...,-2.760365,-1.669572,-2.798357,-1.746826,-1.696603,-1.718214,-1.513312,-1.320840,-1.331297,-1.624776
A2M,2.468968,2.953739,2.763946,2.778267,2.414466,2.519753,2.651989,2.738240,2.794145,2.279002,...,3.103319,2.782747,2.740168,2.322965,2.404073,2.158349,2.198031,2.149877,2.079296,2.015551
A2ML1,0.494442,-0.649893,-0.027800,0.179141,0.201596,-0.890231,-0.527682,0.567013,-0.397934,-0.902149,...,-0.333226,0.015234,0.597015,0.269507,-0.319626,-0.194146,0.016324,0.155709,-0.261940,-0.298371
A3GALT2,-0.722721,-0.710027,-1.000137,-0.473323,-1.424504,0.692757,-0.597187,-0.314624,-0.487444,-0.163617,...,-0.377115,-0.737936,-0.563551,-1.038875,-0.406756,-0.970994,-0.845346,-0.880717,-0.727718,-1.359908


# Normalize Matrix (z-score the rows)

In [42]:
uf.zscore(normalized_matrix, 'row')

In [43]:
normalized_matrix.head()

,TCGA-32-1970-01,TCGA-28-5218-01,TCGA-19-2619-01,TCGA-32-2634-01,TCGA-06-5415-01,TCGA-26-5134-01,TCGA-32-1982-01,TCGA-19-2620-01,TCGA-32-2638-01,TCGA-06-5416-01,...,TCGA-14-0789-01,TCGA-16-1045-01,TCGA-27-1834-01,TCGA-14-2554-01,TCGA-27-2523-01,TCGA-06-0681-11,TCGA-06-0675-11,TCGA-06-0680-11,TCGA-06-AABW-11,TCGA-06-0678-11
A1BG,-1.010202,0.495902,0.149133,-0.666392,1.344042,1.822964,0.360153,0.898074,0.547518,-0.038103,...,0.262492,-0.193743,-0.651129,2.512661,1.029957,-0.344507,-0.519769,-0.619826,-3.731717,-0.499378
A1CF,-0.148557,1.070617,1.006233,0.207741,0.542553,-0.155525,-0.371863,0.025514,-2.510553,1.259844,...,-3.839157,0.051504,-3.974669,-0.224048,-0.044911,-0.121994,0.608855,1.295367,1.258070,0.211284
A2M,-0.634829,0.991874,0.355003,0.403056,-0.817717,-0.464416,-0.020681,0.268743,0.456337,-1.272281,...,1.493806,0.418089,0.275211,-1.124758,-0.852591,-1.677144,-1.543986,-1.705572,-1.942415,-2.156318
A2ML1,1.328033,-1.056227,0.239925,0.671094,0.717880,-1.556979,-0.801597,1.479238,-0.531262,-1.581810,...,-0.396440,0.329586,1.541747,0.859373,-0.368104,-0.106663,0.331858,0.622272,-0.247913,-0.323820
A3GALT2,-0.748829,-0.706691,-1.669738,0.079071,-3.078465,3.949984,-0.332105,0.605887,0.032196,1.107169,...,0.398444,-0.799337,-0.220448,-1.798334,0.300048,-1.572994,-1.155896,-1.273312,-0.765418,-2.864031


# Merge Like Column (by taking the mean)

In [44]:
normalized_matrix = uf.merge(normalized_matrix, 'column', 'mean')

In [45]:
normalized_matrix.shape

(18992, 173)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [46]:
uf.mapgenesymbols(normalized_matrix)

In [47]:
normalized_matrix.shape

(18560, 173)

# Merge Duplicate Genes By Rows

In [48]:
normalized_matrix = uf.merge(normalized_matrix, 'row', 'mean')

In [49]:
normalized_matrix.shape

(18471, 173)

# Histogram of First Sample

In [50]:
sns.distplot(normalized_matrix.iloc[:, 10]);

# Histogram of First Gene

In [51]:
sns.distplot(normalized_matrix.iloc[10, :]);

# Save Filtered Matrix

In [52]:
filename = path+'tcga_gbm_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [53]:
gene_list = uf.createGeneList(normalized_matrix)

In [54]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A3GALT2,127550


In [55]:
gene_list.shape

(18471, 2)

# Save Gene List

In [56]:
filename = path+'tcga_gbm_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute List

In [57]:
metaData = pd.DataFrame(index=normalized_matrix.columns)

for index in metaData.index:
    if ('-').join(index.split('-')[0:3]) in sample_meta.index:
        metaData.loc[index, 'Age'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'age_at_initial_pathologic_diagnosis']
        metaData.loc[index, 'Gender'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'gender']
        metaData.loc[index, 'Pathologic M'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_M']
        metaData.loc[index, 'Pathologic N'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_N']
        metaData.loc[index, 'Pathologic T'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_T']
        metaData.loc[index, 'Pathologic Stage'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_stage']
        metaData.loc[index, 'Histological Type'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'histological_type']
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'tumor_tissue_site']
        metaData.loc[index, 'Neoplasm Histologic Grade'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'neoplasm_histologic_grade']
    else:
        metaData.loc[index, 'Age'] = np.nan
        metaData.loc[index, 'Gender'] = np.nan
        metaData.loc[index, 'Pathologic M'] = np.nan
        metaData.loc[index, 'Pathologic N'] = np.nan
        metaData.loc[index, 'Pathologic T'] = np.nan
        metaData.loc[index, 'Pathologic Stage'] = np.nan
        metaData.loc[index, 'Histological Type'] = np.nan
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = np.nan
        metaData.loc[index, 'Neoplasm Histologic Grade'] = np.nan

In [58]:
attribute_list = uf.createAttributeList(normalized_matrix, metaData)

In [59]:
attribute_list.head()

,Age,Gender,Pathologic M,Pathologic N,Pathologic T,Pathologic Stage,Histological Type,Sample Type,Tumor Tissue Site,Neoplasm Histologic Grade
Attributes,,,,,,,,,,
TCGA-02-0047-01,78,MALE,NaN,NaN,NaN,NaN,Untreated primary (de novo) GBM,Primary Solid Tumor,Brain,NaN
TCGA-02-0055-01,62,FEMALE,NaN,NaN,NaN,NaN,Untreated primary (de novo) GBM,Primary Solid Tumor,Brain,NaN
TCGA-02-2483-01,43,MALE,NaN,NaN,NaN,NaN,Untreated primary (de novo) GBM,Primary Solid Tumor,Brain,NaN
TCGA-02-2485-01,53,MALE,NaN,NaN,NaN,NaN,Untreated primary (de novo) GBM,Primary Solid Tumor,Brain,NaN
TCGA-02-2486-01,64,MALE,NaN,NaN,NaN,NaN,Untreated primary (de novo) GBM,Primary Solid Tumor,Brain,NaN


In [60]:
attribute_list.shape

(173, 10)

# Save Attribute List

In [61]:
filename = path+'tcga_gbm_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', compression='gzip')

# Create matrix of Standardized values (values between -1, and 1)

In [62]:
standard_matrix = uf.createStandardizedMatrix(normalized_matrix)

In [63]:
standard_matrix.head()

,TCGA-02-0047-01,TCGA-02-0055-01,TCGA-02-2483-01,TCGA-02-2485-01,TCGA-02-2486-01,TCGA-06-0125-01,TCGA-06-0125-02,TCGA-06-0129-01,TCGA-06-0130-01,TCGA-06-0132-01,...,TCGA-41-3915-01,TCGA-41-4097-01,TCGA-41-5651-01,TCGA-76-4925-01,TCGA-76-4926-01,TCGA-76-4927-01,TCGA-76-4928-01,TCGA-76-4929-01,TCGA-76-4931-01,TCGA-76-4932-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,-0.373325,0.239263,-0.719866,0.331775,0.968265,-0.003455,-0.061266,0.193077,-0.916290,-0.026650,...,-0.627402,-0.789190,-0.095957,0.586277,0.378089,0.031205,0.482135,-0.130659,0.054350,0.493724
A1CF,-0.954015,-0.064377,-0.295513,0.143715,-0.803865,-0.699871,0.097421,0.814605,-0.561252,-0.133781,...,-0.260871,-0.826947,-0.884697,0.675764,-0.318622,-0.330199,-0.145278,0.826178,0.444438,-0.503476
A2M,0.849571,0.560263,0.016838,-0.745856,0.548740,-0.179705,0.097657,0.710685,0.167091,0.328927,...,0.965395,0.259566,-0.075721,-0.653414,-0.041061,0.629655,0.618050,0.016838,-0.664986,0.063010
A2ML1,-0.278474,-0.936968,-0.913889,0.669651,0.368970,0.264819,0.079849,-0.324690,-0.752138,0.588682,...,0.126041,0.415200,-0.671275,-0.601908,0.959102,0.345791,-0.659721,0.889566,0.577115,0.149240
A3GALT2,-0.001042,0.276346,-0.832978,0.750705,-0.752138,0.079849,-0.336247,0.889566,0.947512,0.195471,...,-0.613480,0.068281,-0.913889,-0.601908,-0.440206,-0.278474,-0.105134,-0.590378,0.877992,-0.705921


# Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [64]:
plt.plot(normalized_matrix[normalized_matrix.columns[0]], standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

# Save Standardized Matrix

In [65]:
filename = path+'tcga_gbm_standard_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
standard_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [66]:
tertiary_matrix = uf.createTertiaryMatrix(standard_matrix)

In [67]:
tertiary_matrix.head()

,TCGA-02-0047-01,TCGA-02-0055-01,TCGA-02-2483-01,TCGA-02-2485-01,TCGA-02-2486-01,TCGA-06-0125-01,TCGA-06-0125-02,TCGA-06-0129-01,TCGA-06-0130-01,TCGA-06-0132-01,...,TCGA-41-3915-01,TCGA-41-4097-01,TCGA-41-5651-01,TCGA-76-4925-01,TCGA-76-4926-01,TCGA-76-4927-01,TCGA-76-4928-01,TCGA-76-4929-01,TCGA-76-4931-01,TCGA-76-4932-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
A2ML1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
A3GALT2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Save Teriary Matrix

In [68]:
filename = path+'tcga_gbm_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Up Gene Set Library

In [69]:
name = 'tcga_gbm_gene_up_set'

In [70]:
uf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [71]:
name = 'tcga_gbm_gene_down_set'

In [72]:
uf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [73]:
name = 'tcga_gbm_attribute_up_set'

In [74]:
uf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [75]:
name = 'tcga_gbm_attribute_down_set'

In [76]:
uf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Attribute Similarity matrix

In [77]:
attribute_normalized_matrix = normalized_matrix.copy()

In [78]:
getSampleData(attribute_normalized_matrix, sample_meta, SampleTypes)

In [79]:
attribute_similarity_matix = uf.createSimilarityMatrix(attribute_normalized_matrix.T, 'cosine')

In [80]:
attribute_similarity_matix.head()

,"(Donor:TCGA-02-0047, Age:78.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-02-0055, Age:62.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-02-2483, Age:43.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-02-2485, Age:53.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-02-2486, Age:64.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-06-0125, Age:63.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-06-0125, Age:63.0, Gender:FEMALE, SampleType:Recurrent Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-06-0129, Age:30.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-06-0130, Age:54.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-06-0132, Age:49.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)",...,"(Donor:TCGA-41-3915, Age:48.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-41-4097, Age:63.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-41-5651, Age:59.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-76-4925, Age:76.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-76-4926, Age:68.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-76-4927, Age:58.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Glioblastoma Multiforme (GBM), Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-76-4928, Age:85.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-76-4929, Age:76.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-76-4931, Age:70.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)","(Donor:TCGA-76-4932, Age:50.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Treated primary GBM, Tissue:Brain, Tumor:nan, Gradenan)"
,,,,,,,,,,,,,,,,,,,,,
"(Donor:TCGA-02-0047, Age:78.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)",1.000000,-0.014788,-0.029588,0.006493,-0.040127,0.025441,0.168965,0.106072,-0.035447,0.116189,...,0.083396,0.115908,0.009341,-0.057435,0.067830,0.021980,-0.034421,0.039828,0.045592,-0.079877
"(Donor:TCGA-02-0055, Age:62.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Untreated primary (de novo) GBM, Tissue:Brain, Tumor:nan, Gradenan)",-0.014788,1.000000,0.117686,-0.031205,0.142385,-0.031398,0.090301,-0.090529,0.293330,0.054445,...,0.

# Save Attribute Similarity Matrix

In [81]:
filename = path+'tcga_gbm_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# HeatMap (clustergrammer) of Similarity Matrix

In [82]:
net.load_df(attribute_similarity_matix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
net.cluster()
net.widget()

# Create Gene Similarity Matrix

In [83]:
gene_similarity_matix = uf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [84]:
gene_similarity_matix.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,-0.106332,-0.203106,0.102368,-0.041091,-0.179100,-0.024852,0.067581,-0.072252,0.197931,...,0.153112,0.126273,0.147221,0.077928,0.119994,0.013370,-0.009512,0.008953,-0.259962,0.180296
A1CF,-0.106332,1.000000,-0.142367,-0.102996,0.097931,-0.081366,0.064178,0.212101,0.138583,-0.034029,...,0.066137,0.061190,0.097709,0.094783,0.022941,0.096911,0.153256,0.001903,0.235821,0.101758
A2M,-0.203106,-0.142367,1.000000,0.008431,-0.034342,0.278825,0.241902,-0.121991,-0.222260,0.022320,...,-0.145173,0.013451,0.115539,-0.021109,-0.078624,-0.076611,-0.319771,0.328266,0.245683,0.155189
A2ML1,0.102368,-0.102996,0.008431,1.000000,-0.034490,-0.059975,-0.110023,-0.137773,-0.031433,-0.011627,...,-0.350688,-0.308182,0.223370,0.138209,0.168758,-0.031433,-0.005955,-0.222019,0.087587,0.055592
A3GALT2,-0.041091,0.097931,-0.034342,-0.034490,1.000000,-0.097088,-0.086106,0.182092,-0.290764,-0.025078,...,-0.052497,0.116595,-0.002772,-0.109236,0.092700,0.085888,-0.104856,0.115056,0.123004,-0.077995


# Save Gene Similarity Matrix

In [85]:
filename = path+'tcga_gbm_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [86]:
name = 'tcga_gbm_gene_attribute_edge_list'

In [87]:
uf.createGeneAttributeEdgeList(standard_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  173 Out of 173   

 The number of statisticaly relevent gene-attribute associations is: 159910
